In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import scipy
from scipy.stats import wilcoxon
import datetime

Get total amount spent in each state for the 2024 election. 

In [8]:
spending_24_df = pd.read_csv("spending_data_2024.csv", index_col=False, low_memory=False)
spending_24_df.groupby('recipient_st').sum('disb_amt').sort_values('disb_amt')
# spending_24_df.head()

,disb_amt,file_num
recipient_st,,
GB,1.799600e+02,7020812
T,2.946500e+02,1751641
UK,5.520000e+02,14085106
QC,5.538000e+02,10522914
C,7.285800e+02,3503282
...,...,...
FL,5.477138e+07,6984054808
CA,6.363736e+07,39228883214
VA,1.432239e+08,15642195573


In [4]:
spending_24_df['recipient_st'].unique()
['AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'FL', 'GA', 'IA', 'IL',
       'IN', 'LA', 'MA', 'MD', 'ME', 'MN', 'MO', 'NC', 'NE', 'NH', 'NJ',
       'NY', 'OK', 'PA', 'SC', 'TN', 'TX', 'VA', 'WA', 'WI', 'WY',
       'OH', 'WV', 'AK', 'DE', 'HI', 'ID', 'KS', 'KY', 'MI',
       'MS', 'MT', 'ND', 'NM', 'NV', 'OR', 'RI', 'UT', 'VT',
       'SD', 'C', 'F', 'G', 'I', 'N',
       'T', 'AA'] 
# C F G I N T AA are errors that need to be replaced with the correct state 

array(['AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'FL', 'GA', 'IA', 'IL',
       'IN', 'LA', 'MA', 'MD', 'ME', 'MN', 'MO', 'NC', 'NE', 'NH', 'NJ',
       'NY', 'OK', 'PA', 'SC', 'TN', 'TX', 'VA', 'VI', 'WA', 'WI', 'WY',
       'ZZ', 'OH', 'WV', 'AK', 'AS', 'DE', 'HI', 'ID', 'KS', 'KY', 'MI',
       'MP', 'MS', 'MT', 'ND', 'NM', 'NV', 'OR', 'RI', 'UT', 'VT', 'GU',
       'PR', 'SD', 'AB', 'GB', 'ON', 'AU', 'C', 'F', 'G', 'I', 'N', 'QC',
       'T', 'TH', 'UK', 'W', 'AA'], dtype=object)